In [23]:
from dotenv import load_dotenv
import os
load_dotenv()
from openai import OpenAI

In [24]:
client = OpenAI(
            api_key=os.getenv("OpenAI_Key"),
        )
model_name = os.getenv("ModelName")

In [25]:
def get_chatbot_response(client,model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content
    
    return response

In [26]:
system_prompt = """
You are an AI assistant specialized in log analysis. Your job is to:
        1. Classify the log level (INFO, WARNING, ERROR, DEBUG).
        2. Summarize the log message.
        3. Identify anomalies or critical issues in the log.
        4. Suggest potential fixes or actions if an issue is detected.
        5. Track the Anamoaly and report it.
        {
            "chainofthought":"Think over the log message and provide the best possible solution",
            "message": "Analyze the log message and provide a summary, classification, and potential issues or fixes.",
        }
        make sure to provide a json structure with the above keys and values.
"""
msg = """ [2025-01-26 16:49:25,078] INFO [GroupCoordinator 0]: Dynamic Member with unknown member id joins group log-consumer-group in Empty state. Created a new member id kafka-python-2.0.2-00970e35-142a-4d7a-8354-8835ac58295a for this member and add to the group. (kafka.coordinator.group.GroupCoordinator)"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':msg})
response = get_chatbot_response(client,model_name,messages)

print(response)

```json
{
    "chainofthought": "The log message indicates that a new dynamic member with an unknown member ID has joined a Kafka consumer group. The system has created a new member ID for this member and added it to the group. This is a normal operation in Kafka when a new consumer joins a group.",
    "message": "Analyze the log message and provide a summary, classification, and potential issues or fixes.",
    "classification": "INFO",
    "summary": "A new dynamic member with an unknown ID has joined the Kafka consumer group 'log-consumer-group'. A new member ID was created and added to the group.",
    "anomalies_or_issues": "No anomalies or critical issues detected. This is a standard operation.",
    "potential_fixes_or_actions": "No action needed as this is a normal process.",
    "anomaly_tracking": "No anomaly detected."
}
```


In [29]:
msg = """ 16.180.70.237 - - [26/Jan/2025:18:05:03 ] "GET /wheelsets HTTP/1.0" 200 2838 "http://bestcyclingreviews.com/top_online_shops" "Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X)" [SERVICE: auth-service | LEVEL: WARNING]
Error extracting JSON from output: local variable 'analysis' referenced before assignment"""

def analyze_log_with_openai(log_message):
    try:
        # System prompt for OpenAI analysis
        system_prompt = """
        You are an AI assistant specialized in log analysis. Your job is to:
        1. Classify the log level (INFO, WARNING, ERROR, DEBUG).
        2. Summarize the log message.
        3. Identify anomalies or critical issues in the log.
        4. Suggest potential fixes or actions if an issue is detected.
        5. Track the Anamoaly and report it.
        {
            "chainofthought":"Think over the log message and provide the best possible solution",
            "message": "Analyze the log message and provide a summary, classification, and potential issues or fixes.",
        }
        make sure to provide a json structure with the above keys and values.
        """
        messages = [{'role':'system','content':system_prompt}]
        messages.append({'role':'user','content':log_message})
        response = get_chatbot_response(client,model_name,messages)
        return response
    except Exception as e:
        return f"Error analyzing log with OpenAI: {e}"

In [32]:
def extract_json_from_output(chatbot_output):
    import re
    import json
    # Enhanced regex to match `json` or `write` after backticks
    json_match = re.search(r'```(?:json|write)\n(.*?)\n```', chatbot_output, re.DOTALL)

    if json_match:
        json_string = json_match.group(1)  # Extract JSON content
        try:
            json_data = json.loads(json_string)  # Convert JSON string to Python dictionary
            return json_string
        except json.JSONDecodeError as e:
            return {"error": f"Failed to decode JSON: {e}"}
    else:
        return {"error": "No JSON found in the input."}

In [33]:
a = analyze_log_with_openai(msg)
out = extract_json_from_output(a)
print(out)

{
    "chainofthought": "The log message indicates that a new dynamic member with an unknown member ID has joined a Kafka consumer group. The system has created a new member ID for this member and added it to the group.",
    "message": "Analyze the log message and provide a summary, classification, and potential issues or fixes.",
    "classification": "INFO",
    "summary": "A new dynamic member with an unknown ID has joined the Kafka consumer group 'log-consumer-group'. A new member ID was created and added to the group.",
    "anomalies_or_issues": "No anomalies or critical issues detected. This is a standard informational log indicating normal operation.",
    "potential_fixes_or_actions": "No action required as this is an informational message indicating normal behavior.",
    "anomaly_tracking": "No anomaly detected in this log entry."
}


In [35]:
import pymongo
from datetime import datetime

In [38]:
def analyze_log_with_openai(client,model_name,log_message):
    try:
        system_prompt = """
        You are an AI assistant specialized in log analysis. Your job is to:
        1. Classify the log level (INFO, WARNING, ERROR, DEBUG).
        2. Summarize the log message.
        3. Identify anomalies or critical issues in the log.
        4. Suggest potential fixes or actions if an issue is detected.
        5. Track the Anamoaly and report it.
        {
            "chainofthought":"Think over the log message and provide the best possible solution",
            "message": "Analyze the log message and provide a summary, classification, and potential issues or fixes.",
        }
        make sure to provide a json structure with the above keys and values.
        """
        messages = [{'role':'system','content':system_prompt}]
        messages.append({'role':'user','content':log_message})
        print("messages",messages)
        response = get_chatbot_response(client,model_name,messages)
        return response
    except Exception as e:
        return f"Error analyzing log with OpenAI: {e}"
    
def get_chatbot_response(client,model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content
    print("response",response)
    return response

def extract_json_from_output(chatbot_output):
    import re
    import json
    # Enhanced regex to match `json` or `write` after backticks
    json_match = re.search(r'```(?:json|write)\n(.*?)\n```', chatbot_output, re.DOTALL)

    if json_match:
        json_string = json_match.group(1)  # Extract JSON content
        try:
            json_data = json.loads(json_string)  # Convert JSON string to Python dictionary
            return json_string
        except json.JSONDecodeError as e:
            return {"error": f"Failed to decode JSON: {e}"}
    else:
        return {"error": "No JSON found in the input."}

In [41]:

def main():

    client = OpenAI(
                api_key=os.getenv("OpenAI_Key"),
            )
    model_name = os.getenv("ModelName")
    log_message = """ 16.180.70.237 - - [26/Jan/2025:18:05:03 ] "GET /wheelsets HTTP/1.0" 200 2838 "http://bestcyclingreviews.com/top_online_shops" "Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X)" [SERVICE: auth-service | LEVEL: WARNING]
                    Error extracting JSON from output: local variable 'analysis' referenced before assignment"""

    analysis = analyze_log_with_openai(client,model_name,log_message)
    print(f"AI Analysis:\n{analysis}")
    try:
        print("extract json")
        analysis = extract_json_from_output(analysis)
    except Exception as e:
        print(f"Error extracting JSON from output: {e}")
        pass
    print(f"AI Analysis:\n{analysis}")

    # Store log and analysis in MongoDB
    log_entry = {
        "log_message": log_message,
        "analysis": analysis,
        "timestamp": datetime.now()
    }
    logs_collection.insert_one(log_entry)
    print(f"Stored log in MongoDB: {log_entry}")

In [42]:
main()

messages [{'role': 'system', 'content': '\n        You are an AI assistant specialized in log analysis. Your job is to:\n        1. Classify the log level (INFO, WARNING, ERROR, DEBUG).\n        2. Summarize the log message.\n        3. Identify anomalies or critical issues in the log.\n        4. Suggest potential fixes or actions if an issue is detected.\n        5. Track the Anamoaly and report it.\n        {\n            "chainofthought":"Think over the log message and provide the best possible solution",\n            "message": "Analyze the log message and provide a summary, classification, and potential issues or fixes.",\n        }\n        make sure to provide a json structure with the above keys and values.\n        '}, {'role': 'user', 'content': ' 16.180.70.237 - - [26/Jan/2025:18:05:03 ] "GET /wheelsets HTTP/1.0" 200 2838 "http://bestcyclingreviews.com/top_online_shops" "Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X)" [SERVICE: auth-service | LEVEL: WARNING]\n                

NameError: name 'logs_collection' is not defined